In [ ]:
# Oznacio oblast na mapi GFW-a
# Skinuti podatke za tree cover loss
# Skinuti podatke za tree cover by type
# Sacuvati podatke lokalno
# I posle 2 nedelje nemam opciju da skinem podatke?? a pise na GFW-u od 12-24h da se ceka za podatke 

# Ucitavanje podataka

import rasterio
import matplotlib.pyplot as plt

# Putanja do skinutih GeoTIFF fajlova
file_path_loss = os.path.join(os.getcwd(), 'treecover_loss')
file_path_type = os.path.join(os.getcwd(), 'treecover_type')

with rasterio.open(file_path_loss) as src:
    tree_cover_loss = src.read(1)  
    profile = src.profile

df = pd.DataFrame(tree_cover_loss, columns=['TreeCoverLoss'])

# Vizualizacija podataka
plt.imshow(tree_cover_loss , cmap='RdYlGn', vmin=0, vmax=1)  
plt.colorbar()
plt.title('Tree Cover Loss')
plt.show()

In [ ]:
#Preprocesing

import json
from rasterio.mask import mask

aoi_geojson = {
    "type": "Polygon",
    "coordinates": [
        [
            [20.611538, 43.045487],
            [20.619155, 43.045487],
            [20.619155, 43.051737],
            [20.611538, 43.051737],
            [20.611538, 43.045487]  # Jos jedna tacka za zatvaranje poligona, kao na Global Forest Watchu
        ]
    ]
}

with open('aoi.geojson', 'w') as f:
    json.dump(aoi_geojson, f)
    

with rasterio.open(file_path_loss) as src:
    out_image, out_transform = mask(src, [aoi_geojson], crop=True)
    out_meta = src.meta.copy()

# Promena metapodataka za klipovanu sliku
out_meta.update({
    "driver": "GTiff",
    "count": 1,
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Cuvanje klipovane slike
clipped_file_path = os.path.join(os.getcwd(), 'clipped_tree_cover_loss.tiff')
with rasterio.open(clipped_file_path, "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
import rasterio
import pandas as pd
import matplotlib.pyplot as plt

# Ucitavanje klipovane NDVI slike
with rasterio.open(clipped_file_path) as src:
    tree_cover_loss_data = src.read(1) 
    profile = src.profile

df = pd.DataFrame(tree_cover_loss_data)

df_flat = df.stack().reset_index()
df_flat.columns = ['row', 'col', 'TreeCoverLoss']

threshold = 10
df_flat['Label'] = np.where(df_flat['TreeCoverLoss'] > threshold, 1, 0)
